In [1]:
import pandas as pd
pd.set_option("display.max_columns",250)
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import xlsxwriter
import yaml as yaml

sns.set()

In [2]:
with open(r'E:/Projects/Clients/MetCouncilASIM/tasks/survey_data_processing/metc_inputs.yml') as file:
    settings = yaml.full_load(file)

output_path = r"output"

processed_transit_survey_path = r"output"
spa_input_path = os.path.join(settings['proj_dir'], 'SPA_Inputs')
spa_output_path = os.path.join(settings['proj_dir'], 'SPA_Processed')

tour_mode_choice_output_path = os.path.join(output_path, 'tour_mode_choice')
trip_mode_choice_output_path = os.path.join(output_path, 'trip_mode_choice')

In [3]:
# Reading in tables
obs_df = pd.read_csv(os.path.join(processed_transit_survey_path, 'processed_obs.csv'))

spa_in_hh_df = pd.read_csv(os.path.join(spa_input_path, "HH_SPA_INPUT.csv"), encoding='latin1')
spa_in_per_df = pd.read_csv(os.path.join(spa_input_path, "PER_SPA_INPUT.csv"), encoding='latin1')
spa_in_place_1 =  pd.read_csv(os.path.join(spa_input_path, "place_1.csv"), encoding='latin1')
spa_in_place_2 =  pd.read_csv(os.path.join(spa_input_path, "place_2.csv"), encoding='latin1')
spa_in_place_3 =  pd.read_csv(os.path.join(spa_input_path, "place_3.csv"), encoding='latin1')
spa_in_place_4 =  pd.read_csv(os.path.join(spa_input_path, "place_4.csv"), encoding='latin1')

spa_in_place_df = spa_in_place_1.append([spa_in_place_2, spa_in_place_3, spa_in_place_4])

for dayno in range(1, 5):
    print(dayno)
    spa_out_per_filename = os.path.join(spa_output_path, "day" + str(dayno), "persons.csv")
    spa_out_per_df_day =  pd.read_csv(spa_out_per_filename, encoding='latin1')
    spa_out_tours_filename = os.path.join(spa_output_path, "day" + str(dayno), "tours.csv")
    spa_out_tours_df_day =  pd.read_csv(spa_out_tours_filename, encoding='latin1')
    spa_out_trips_filename = os.path.join(spa_output_path, "day" + str(dayno), "trips.csv")
    spa_out_trips_df_day =  pd.read_csv(spa_out_trips_filename, encoding='latin1')
    
    spa_out_trips_df_day['DOW'] = dayno
    spa_out_tours_df_day['DOW'] = dayno
                            
    if dayno == 1:
        spa_out_per_df = spa_out_per_df_day.copy()
        spa_out_tours_df = spa_out_tours_df_day.copy()
        spa_out_trips_df = spa_out_trips_df_day.copy()
    else:
        spa_out_per_df = spa_out_per_df.append(spa_out_per_df_day)
        spa_out_tours_df = spa_out_tours_df.append(spa_out_tours_df_day)        
        spa_out_trips_df = spa_out_trips_df.append(spa_out_trips_df_day)        

# warnings on column types...
spa_out_per_df = spa_out_per_df.drop(columns = ['ERROR'])
spa_out_per_df = spa_out_per_df.drop_duplicates()



C:\Users\leah.flake\.conda\envs\mcasim\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1


C:\Users\leah.flake\.conda\envs\mcasim\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (219,228) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2
3
4


In [4]:
output_excel_path = os.path.join(output_path, 'calibration_target_tables.xlsx')
excel_writer = pd.ExcelWriter(output_excel_path, engine='xlsxwriter')

# Matching mode  and purpose names between surveys

In [5]:
output_calibration_modes = ['SOV','HOV2','HOV3', 'WALK', 'BIKE', 'WALK-TRANSIT',
                            'PNR-TRANSIT','KNR-TRANSIT', 'TAXI', 'TNC-SINGLE', 'TNC-SHARED','SCHOOLBUS', 'All']
output_calibration_purposes = ['Work', 'University', 'School', 'Work sub-tour', 'Ind-Maintenance', 'Ind-Discretionary',
                               'Joint-Maintenance', 'Joint-Discretionary',  'All']

In [6]:
spa_in_place_df
spa_out_trips_df['DOW'].value_counts()


2    44299
4    43478
3    42973
1    36819
Name: DOW, dtype: int64

#### Household Travel Survey

In [7]:
spa_in_place_df['TRIPNO'] = spa_in_place_df['PLANO'] -1

spa_out_trips_df = pd.merge(spa_out_trips_df, spa_in_place_df[['SAMPN', 'TRIPNO', 'PERNO', 'DOW', 'TRIP_WEIGHT']],
                            left_on  = ['HH_ID', 'PER_ID', 'TRIP_ID', 'DOW'], right_on = ['SAMPN', 'PERNO', 'TRIPNO', 'DOW'])

# expansion factor already includes the weight
# spa_out_trips_df["PER_WEIGHT"] = spa_out_trips_df['SUBREGCluster_balanced_weight']
spa_out_trips_df.TRIP_WEIGHT.sum()

15015545.94454128

In [8]:
tour_weights = spa_out_trips_df.groupby(['HH_ID', 'PER_ID', 'DOW', 'TOUR_ID'])['TRIP_WEIGHT'].agg('mean').reset_index()
spa_out_tours_df = pd.merge(spa_out_tours_df, tour_weights, on = ['HH_ID', 'PER_ID', 'DOW', 'TOUR_ID'])

spa_out_tours_df

,HH_ID,PER_ID,TOUR_ID,ORIG_PLACENO,DEST_PLACENO,ORIG_X,ORIG_Y,ORIG_TAZ,ORIG_MAZ,DEST_X,DEST_Y,DEST_TAZ,DEST_MAZ,DEST_MODE,ORIG_MODE,TOURPURP,TOURMODE,DRIVER,ANCHOR_DEPART_HOUR,ANCHOR_DEPART_MIN,ANCHOR_DEPART_BIN,PRIMDEST_ARRIVE_HOUR,PRIMDEST_ARRIVE_MIN,PRIMDEST_ARRIVE_BIN,PRIMDEST_DEPART_HOUR,PRIMDEST_DEPART_MIN,PRIMDEST_DEPART_BIN,ANCHOR_ARRIVE_HOUR,ANCHOR_ARRIVE_MIN,ANCHOR_ARRIVE_BIN,TOUR_DUR_HR,TOUR_DUR_MIN,TOUR_DUR_BIN,MAJOR_UNIV_DEST,SPEC_EVENT_DEST,IS_SUBTOUR,PARENT_TOUR_ID,PARENT_TOUR_MODE,NUM_SUBTOURS,CHILD_TOUR_ID_1,CHILD_TOUR_ID_2,CHILD_TOUR_ID_3,ESCORTED_TOUR,CHAUFFUER_ID,ESCORTING_TOUR,NUM_PERSONS_ESCORTED,ESCORT_PERS_1,ESCORT_PERS_2,ESCORT_PERS_3,ESCORT_PERS_4,ESCORT_PERS_5,OUTBOUND_STOPS,INBOUND_STOPS,OSTOP_1_PLACENO,OSTOP_1_X,OSTOP_1_Y,OSTOP_1_TAZ,OSTOP_1_MAZ,OSTOP_1_ARR_HR,OSTOP_1_ARR_MIN,OSTOP_1_ARR_BIN,OSTOP_1_DEP_HR,OSTOP_1_DEP_MIN,OSTOP_1_DEP_BIN,OSTOP_1_DUR_HR,OSTOP_1_DUR_MIN,OSTOP_1_DUR_BIN,OSTOP_1_PURP,OSTOP_1_MODE,OSTOP_1_ESCORT_ID,OSTOP_1_PUDO,OSTOP_1_MAJUNIV,OSTOP_1_SPECEVENT,OSTOP_2_PLACENO,OSTOP_2_X,OSTOP_2_Y,OSTOP_2_TAZ,OSTOP_2_MAZ,OSTOP_2_ARR_HR,OSTOP_2_ARR_MIN,OSTOP_2_ARR_BIN,OSTOP_2_DEP_HR,OSTOP_2_DEP_MIN,OSTOP_2_DEP_BIN,OSTOP_2_DUR_HR,OSTOP_2_DUR_MIN,OSTOP_2_DUR_BIN,OSTOP_2_PURP,OSTOP_2_MODE,OSTOP_2_ESCORT_ID,OSTOP_2_PUDO,OSTOP_2_MAJUNIV,OSTOP_2_SPECEVENT,OSTOP_3_PLACENO,OSTOP_3_X,OSTOP_3_Y,OSTOP_3_TAZ,OSTOP_3_MAZ,OSTOP_3_ARR_HR,OSTOP_3_ARR_MIN,OSTOP_3_ARR_BIN,OSTOP_3_DEP_HR,OSTOP_3_DEP_MIN,OSTOP_3_DEP_BIN,OSTOP_3_DUR_HR,OSTOP_3_DUR_MIN,OSTOP_3_DUR_BIN,OSTOP_3_PURP,OSTOP_3_MODE,OSTOP_3_ESCORT_ID,OSTOP_3_PUDO,OSTOP_3_MAJUNIV,OSTOP_3_SPECEVENT,OSTOP_4_PLACENO,OSTOP_4_X,OSTOP_4_Y,OSTOP_4_TAZ,OSTOP_4_MAZ,OSTOP_4_ARR_HR,OSTOP_4_ARR_MIN,OSTOP_4_ARR_BIN,OSTOP_4_DEP_HR,OSTOP_4_DEP_MIN,OSTOP_4_DEP_BIN,OSTOP_4_DUR_HR,OSTOP_4_DUR_MIN,OSTOP_4_DUR_BIN,OSTOP_4_PURP,OSTOP_4_MODE,OSTOP_4_ESCORT_ID,OSTOP_4_PUDO,OSTOP_4_MAJUNIV,OSTOP_4_SPECEVENT,ISTOP_1_PLACENO,ISTOP_1_X,ISTOP_1_Y,ISTOP_1_TAZ,ISTOP_1_MAZ,ISTOP_1_ARR_HR,ISTOP_1_ARR_MIN,ISTOP_1_ARR_BIN,ISTOP_1_DEP_HR,ISTOP_1_DEP_MIN,ISTOP_1_DEP_BIN,ISTOP_1_DUR_HR,ISTOP_1_DUR_MIN,ISTOP_1_DUR_BIN,ISTOP_1_PURP,ISTOP_1_MODE,ISTOP_1_ESCORT_ID,ISTOP_1_PUDO,ISTOP_1_MAJUNIV,ISTOP_1_SPECEVENT,ISTOP_2_PLACENO,ISTOP_2_X,ISTOP_2_Y,ISTOP_2_TAZ,ISTOP_2_MAZ,ISTOP_2_ARR_HR,ISTOP_2_ARR_MIN,ISTOP_2_ARR_BIN,ISTOP_2_DEP_HR,ISTOP_2_DEP_MIN,ISTOP_2_DEP_BIN,ISTOP_2_DUR_HR,ISTOP_2_DUR_MIN,ISTOP_2_DUR_BIN,ISTOP_2_PURP,ISTOP_2_MODE,ISTOP_2_ESCORT_ID,ISTOP_2_PUDO,ISTOP_2_MAJUNIV,ISTOP_2_SPECEVENT,ISTOP_3_PLACENO,ISTOP_3_X,ISTOP_3_Y,ISTOP_3_TAZ,ISTOP_3_MAZ,ISTOP_3_ARR_HR,ISTOP_3_ARR_MIN,ISTOP_3_ARR_BIN,ISTOP_3_DEP_HR,ISTOP_3_DEP_MIN,ISTOP_3_DEP_BIN,ISTOP_3_DUR_HR,ISTOP_3_DUR_MIN,ISTOP_3_DUR_BIN,ISTOP_3_PURP,ISTOP_3_MODE,ISTOP_3_ESCORT_ID,ISTOP_3_PUDO,ISTOP_3_MAJUNIV,ISTOP_3_SPECEVENT,ISTOP_4_PLACENO,ISTOP_4_X,ISTOP_4_Y,ISTOP_4_TAZ,ISTOP_4_MAZ,ISTOP_4_ARR_HR,ISTOP_4_ARR_MIN,ISTOP_4_ARR_BIN,ISTOP_4_DEP_HR,ISTOP_4_DEP_MIN,ISTOP_4_DEP_BIN,ISTOP_4_DUR_HR,ISTOP_4_DUR_MIN,ISTOP_4_DUR_BIN,ISTOP_4_PURP,ISTOP_4_MODE,ISTOP_4_ESCORT_ID,ISTOP_4_PUDO,ISTOP_4_MAJUNIV,ISTOP_4_SPECEVENT,PEREXPFACT,HHEXPFACT,PERSONTYPE,FULLY_JOINT,PARTIAL_TOUR,JTOUR_ID,ERROR,JOINT_STATUS,JOINT_TOUR_PURP,DIST,OUT_ESCORT_TYPE,OUT_CHAUFFUER_ID,OUT_CHAUFFUER_PURP,OUT_CHAUFFUER_PTYPE,INB_ESCORT_TYPE,INB_CHAUFFUER_ID,INB_CHAUFFUER_PURP,INB_CHAUFFUER_PTYPE,OUT_ESCORTING_TYPE,INB_ESCORTING_TYPE,OUT_ESCORTEE_TOUR_PURP,INB_ESCORTEE_TOUR_PURP,OUT_ESCORTING_EPISODES,INB_ESCORTING_EPISODES,DOW,TRIP_WEIGHT
0,18114244,1,1,1.0,2,-93.28638,45.01417,1191,NaN,-93.31758,45.05689,1105,NaN,2.0,2.0,5,2,0,15.0,36.0,26.0,15.0,51.0,26.0,18.0,29.0,31.0,19.0,13.0,33.0,3.0,37.0,8.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [9]:
spa_out_tours_df['TOUR_WEIGHT'] = spa_out_tours_df['TRIP_WEIGHT']

In [10]:
adults_df = spa_in_per_df[spa_in_per_df['EMPLY'].isin(range(1, 2 + 1))]['SAMPN'].value_counts().reset_index()
adults_df.columns=['SAMPN', 'num_workers']

print(adults_df.groupby('num_workers').count())


             SAMPN
num_workers       
1             2928
2             2494
3              212
4               43
5                5


In [11]:
# Calculating auto sufficiency for households

spa_in_hh_df.drop(columns=['num_workers'], errors = 'ignore')
spa_in_hh_df = pd.merge(
    spa_in_hh_df,
    adults_df[['SAMPN', 'num_workers']],
    how='left',
    left_on=['SAMPN'],
    right_on=['SAMPN']
)



In [12]:
spa_in_hh_df['auto_suff'] = 0

spa_in_hh_df.loc[pd.isnull(spa_in_hh_df.num_workers), 'num_workers']=0
spa_in_hh_df.loc[((spa_in_hh_df['HH_VEH_CAT'] < spa_in_hh_df['num_workers']) 
                     & (spa_in_hh_df['HH_VEH_CAT'] > 0)),
                    'auto_suff'] = 1
spa_in_hh_df.loc[((spa_in_hh_df['HH_VEH_CAT'] >= spa_in_hh_df['num_workers']) 
                     & (spa_in_hh_df['HH_VEH_CAT'] > 0)),
                    'auto_suff'] = 2



In [13]:
spa_in_hh_df[['SAMPN', 'auto_suff']]

,SAMPN,auto_suff
0,18112062,2
1,18114244,0
2,18114304,2
3,18114536,2
4,18114695,2
...,...,...
7832,19998608,2
7833,19999119,2
7834,19999446,2
7835,19999572,2


In [14]:
# Merging auto suffiency from Households
spa_out_trips_df = pd.merge(
    spa_out_trips_df,
    spa_in_hh_df[['SAMPN', 'auto_suff']],
    how='left',
    left_on=['HH_ID'],
    right_on=['SAMPN']
)

spa_out_tours_df = pd.merge(
    spa_out_tours_df,
    spa_in_hh_df[['SAMPN', 'auto_suff']],
    how='left',
    left_on=['HH_ID'],
    right_on=['SAMPN']
)


In [15]:
spa_out_trips_df.DOW.value_counts()

2    44299
4    43478
3    42973
1    36819
Name: DOW, dtype: int64

In [16]:
# get tour weight


spa_out_tours_df["HH_WEIGHT"] = spa_out_tours_df['TOUR_WEIGHT']
spa_out_tours_df["PER_WEIGHT"] = spa_out_tours_df['TOUR_WEIGHT']


In [17]:

spa_out_trips_df["HH_WEIGHT"] = spa_out_trips_df['TRIP_WEIGHT']
spa_out_trips_df["PER_WEIGHT"] = spa_out_trips_df['TRIP_WEIGHT']


In [18]:
# Merging joint status and subtour status from tours to trips
spa_out_trips_df = pd.merge(
    spa_out_trips_df,
    spa_out_tours_df[['HH_ID', 'PER_ID', 'TOUR_ID', 'JOINT_STATUS', 'IS_SUBTOUR', 'HH_WEIGHT', 'DOW']],
    how='left',
    on=['HH_ID', 'PER_ID', 'TOUR_ID', 'DOW']
)
spa_out_trips_df.DOW.value_counts()


2    44299
4    43478
3    42973
1    36819
Name: DOW, dtype: int64

#### Removing duplicate joint trips and tours

In [19]:
spa_out_tours_df.loc[spa_out_tours_df['JOINT_STATUS'] == 3, 'PER_WEIGHT'].sum()

715517.2870108483

In [20]:

joint_tours_df = spa_out_tours_df[spa_out_tours_df['JOINT_STATUS'] == 3]
print("Unweighted number of joint tours: ", len(joint_tours_df))


# selecting the tour from the first person in the tour file
unique_joint_tours_df = joint_tours_df.groupby(['HH_ID', 'TOUR_ID', 'DOW']).first().reset_index()
unique_joint_tours_df['unique_joint_tour'] = 1
print("Unweighted number of unique joint tours: ", len(unique_joint_tours_df))


Unweighted number of joint tours:  7191
Unweighted number of unique joint tours:  4014


In [21]:
spa_out_tours_df_backup = spa_out_tours_df.copy()

In [22]:
# spa_out_tours_df = spa_out_tours_df_backup.copy()

In [23]:

# denoting the joint tour to keep
spa_out_tours_df = pd.merge(
    spa_out_tours_df,
    unique_joint_tours_df[['HH_ID', 'PER_ID', 'TOUR_ID', 'DOW', 'unique_joint_tour']],
    how='left',
    on=['HH_ID', 'PER_ID', 'TOUR_ID', 'DOW']
)

# removing non-unique joint tours
spa_out_tours_df = spa_out_tours_df[
    (spa_out_tours_df['JOINT_STATUS'] != 3)
    | ((spa_out_tours_df['JOINT_STATUS'] == 3) & (spa_out_tours_df['unique_joint_tour'] == 1))]


assert len(spa_out_tours_df[spa_out_tours_df['JOINT_STATUS'] == 3]) == len(unique_joint_tours_df), "Joint tours not unique!"

In [24]:
# fully joint trips are joint_status == 3
joint_trips_df = spa_out_trips_df[(spa_out_trips_df['FULLY_JOINT'] == 1)]
print("Unweighted number of joint trips: ", len(joint_trips_df))

# selecting the tour from the first person in the tour file
unique_joint_trips_df = joint_trips_df.groupby(['HH_ID', 'TOUR_ID', 'TRIP_ID', 'DOW']).first().reset_index()
unique_joint_trips_df['unique_joint_trip'] = 1
print("Unweighted number of unique joint trips: ", len(unique_joint_trips_df))

# denoting the joint tour to keep
spa_out_trips_df = pd.merge(
    spa_out_trips_df,
    unique_joint_trips_df[['HH_ID', 'PER_ID', 'TOUR_ID', 'TRIP_ID', 'DOW', 'unique_joint_trip']],
    how='left',
    on=['HH_ID', 'PER_ID', 'TOUR_ID', 'TRIP_ID', 'DOW']
)
# removing non-unique joint trips. All joint trips should belong to a fully joint tour (checked implicity in assert)
spa_out_trips_df = spa_out_trips_df[
    (spa_out_trips_df['JOINT_STATUS'] != 3)
    | ((spa_out_trips_df['JOINT_STATUS'] == 3) & (spa_out_trips_df['unique_joint_trip'] == 1))]



Unweighted number of joint trips:  18375
Unweighted number of unique joint trips:  10206


#### Mapping spa purpose codes to calibration purpose categories

In [25]:
spa_purp_dict = {
    0: 'HOME',
    1: 'WORK',
    2: 'UNIVERSITY',
    3: 'SCHOOL',
    4: 'ESCORTING',
    5: 'SHOPPING',
    6: 'MAINTENANCE',
    7: 'EAT OUT',
    8: 'SOCIAL/VISIT',
    9: 'DISCRETIONARY',
    10: 'WORK-RELATED',
    11: 'LOOP',
    12: 'CHANGE MODE',
    13: 'OTHER',
}
spa_out_trips_df['spa_tour_purpose'] = spa_out_trips_df['TOURPURP'].apply(lambda x: spa_purp_dict[x])

spa_to_abms_purp_dict = {
     'HOME': 'Home',
     'WORK': 'Work',
     'UNIVERSITY': 'University',
     'SCHOOL': 'School',
     'ESCORTING': 'Maintenance',
     'SHOPPING': 'Maintenance',
     'MAINTENANCE': 'Maintenance',
     'EAT OUT': 'Discretionary',
     'SOCIAL/VISIT': 'Discretionary',
     'DISCRETIONARY': 'Discretionary',
     'WORK-RELATED': 'Maintenance',
     'LOOP': 'Discretionary',
     'CHANGE MODE': 'Change Mode',
     'OTHER': 'Discretionary',
}
spa_out_trips_df['tour_purpose'] = spa_out_trips_df['spa_tour_purpose'].apply(lambda x: spa_to_abms_purp_dict[x])
spa_out_trips_df.loc[spa_out_trips_df['IS_SUBTOUR'] == 1, 'tour_purpose'] = 'Work sub-tour'

In [26]:
spa_out_trips_df['tour_purpose'].value_counts(dropna=False)

Maintenance      70085
Discretionary    40259
Work             34798
School            7058
Work sub-tour     6231
University         774
Change Mode        195
Name: tour_purpose, dtype: int64

In [27]:
def group_tour_purposes(tour_purpose):
    if tour_purpose in ['Work', 'University', 'School', 'Work sub-tour']:
        return tour_purpose
    if tour_purpose in ['Escorting','Shopping','Maintenance']:
        return 'Ind-Maintenance'
    if tour_purpose in ['Social/Rec', 'Eat Out', 'Discretionary', 'Home', 'Change Mode']:
        return 'Ind-Discretionary'
    return 'Ind-Discretionary'

spa_out_trips_df['grouped_tour_purpose'] = spa_out_trips_df['tour_purpose'].apply(lambda x: group_tour_purposes(x))

# joint status of 3 is fully joint tour
spa_out_trips_df.loc[(spa_out_trips_df['grouped_tour_purpose'] == 'Ind-Maintenance') & (spa_out_trips_df['JOINT_STATUS'] == 3),
                 'grouped_tour_purpose'] = 'Joint-Maintenance'
spa_out_trips_df.loc[(spa_out_trips_df['grouped_tour_purpose'] == 'Ind-Discretionary') & (spa_out_trips_df['JOINT_STATUS'] == 3),
                 'grouped_tour_purpose'] = 'Joint-Discretionary'
spa_out_trips_df['grouped_tour_purpose'].value_counts(dropna=False)

Ind-Maintenance        65011
Ind-Discretionary      35851
Work                   34798
School                  7058
Work sub-tour           6231
Joint-Maintenance       5074
Joint-Discretionary     4603
University               774
Name: grouped_tour_purpose, dtype: int64

In [28]:
spa_out_trips_df['TRIPMODE'].value_counts(dropna=False).sort_index(level = 'TRIPMODE')

len(spa_out_trips_df)

159400

In [29]:
spa_mode_dict = {
    1: 'SOV',
    2: 'HOV2',
    3: 'HOV3',
    4: 'WALK',
    5: 'BIKE',
    6: 'WALK-LOCAL',
    7: 'WALK-PREMIUM',
    8: 'WALK-MIXED',
    9: 'PNR-LOCAL',
    10: 'PNR-PREMIUM',
    11: 'PNR-MIXED',
    12: 'KNR-LOCAL',
    13: 'KNR-PREMIUM',
    14: 'KNR-MIXED',
    15: 'TNC-SINGLE',
    16: 'TNC-SHARED',
    17: 'TAXI',
    18: 'SCHOOLBUS',
    19: 'OTHER'
}


spa_out_trips_df['linked_trip_mode'] = spa_out_trips_df['TRIPMODE'].apply(lambda x: spa_mode_dict[x])
spa_out_trips_df['tour_mode'] = spa_out_trips_df['TOURMODE'].apply(lambda x: spa_mode_dict[x])

In [30]:
print(spa_out_trips_df['tour_mode'].value_counts(dropna=False))
print(spa_out_trips_df['linked_trip_mode'].value_counts(dropna=False))


SOV             67300
HOV2            36227
HOV3            29755
WALK            10962
WALK-LOCAL       3725
WALK-PREMIUM     3188
SCHOOLBUS        3058
BIKE             2109
PNR-LOCAL         791
OTHER             528
PNR-PREMIUM       440
WALK-MIXED        348
TNC-SINGLE        319
KNR-LOCAL         275
KNR-PREMIUM       171
TAXI              148
PNR-MIXED          27
TNC-SHARED         15
KNR-MIXED          14
Name: tour_mode, dtype: int64
SOV             78338
HOV2            31085
HOV3            20839
WALK            16678
WALK-LOCAL       2882
SCHOOLBUS        2514
BIKE             2311
WALK-PREMIUM     1696
OTHER            1642
TNC-SINGLE        407
PNR-LOCAL         353
WALK-MIXED        166
PNR-PREMIUM       165
TAXI              136
KNR-LOCAL          95
KNR-PREMIUM        61
TNC-SHARED         11
KNR-MIXED          11
PNR-MIXED          10
Name: linked_trip_mode, dtype: int64


In [31]:
def group_transit_modes(df, mode):
    series_name = 'grouped_' + mode
    df[series_name] = df[mode]
    df.loc[(df[mode] == 'WALK-LOCAL') |(df[mode] == 'WALK-PREMIUM')|(df[mode] == 'WALK-MIXED'), series_name] = 'WALK-TRANSIT'
    df.loc[(df[mode] == 'PNR-LOCAL') |(df[mode] == 'PNR-PREMIUM')|(df[mode] == 'PNR-MIXED'), series_name] = 'PNR-TRANSIT'
    df.loc[(df[mode] == 'KNR-LOCAL') |(df[mode] == 'KNR-PREMIUM')|(df[mode] == 'KNR-MIXED'), series_name] = 'KNR-TRANSIT'
    return df

In [32]:
spa_out_trips_df = group_transit_modes(df=spa_out_trips_df, mode='tour_mode')
spa_out_trips_df = group_transit_modes(df=spa_out_trips_df, mode='linked_trip_mode')




In [33]:
print(spa_out_trips_df['grouped_linked_trip_mode'].value_counts(dropna=False))
print(spa_out_trips_df['grouped_tour_mode'].value_counts(dropna=False))
print(spa_out_trips_df['linked_trip_mode'].value_counts(dropna=False))

SOV             78338
HOV2            31085
HOV3            20839
WALK            16678
WALK-TRANSIT     4744
SCHOOLBUS        2514
BIKE             2311
OTHER            1642
PNR-TRANSIT       528
TNC-SINGLE        407
KNR-TRANSIT       167
TAXI              136
TNC-SHARED         11
Name: grouped_linked_trip_mode, dtype: int64
SOV             67300
HOV2            36227
HOV3            29755
WALK            10962
WALK-TRANSIT     7261
SCHOOLBUS        3058
BIKE             2109
PNR-TRANSIT      1258
OTHER             528
KNR-TRANSIT       460
TNC-SINGLE        319
TAXI              148
TNC-SHARED         15
Name: grouped_tour_mode, dtype: int64
SOV             78338
HOV2            31085
HOV3            20839
WALK            16678
WALK-LOCAL       2882
SCHOOLBUS        2514
BIKE             2311
WALK-PREMIUM     1696
OTHER            1642
TNC-SINGLE        407
PNR-LOCAL         353
WALK-MIXED        166
PNR-PREMIUM       165
TAXI              136
KNR-LOCAL          95
KNR-PREMIUM    

In [34]:
spa_out_tours_df['tour_mode'] = spa_out_tours_df['TOURMODE'].apply(lambda x: spa_mode_dict[x])
spa_out_tours_df = group_transit_modes(df=spa_out_tours_df, mode='tour_mode')


In [35]:
spa_out_tours_df['spa_tour_purpose'] = spa_out_tours_df['TOURPURP'].apply(lambda x: spa_purp_dict[x])
spa_out_tours_df['tour_purpose'] = spa_out_tours_df['spa_tour_purpose'].apply(lambda x: spa_to_abms_purp_dict[x])
spa_out_tours_df.loc[spa_out_tours_df['IS_SUBTOUR'] == 1, 'tour_purpose'] = 'Work sub-tour'

spa_out_tours_df['grouped_tour_purpose'] = spa_out_tours_df['tour_purpose'].apply(lambda x: group_tour_purposes(x))
# joint status of 3 is fully joint tour
spa_out_tours_df.loc[(spa_out_tours_df['grouped_tour_purpose'] == 'Ind-Maintenance')
                      & (spa_out_tours_df['JOINT_STATUS'] == 3),
                     'grouped_tour_purpose'] = 'Joint-Maintenance'
spa_out_tours_df.loc[(spa_out_tours_df['grouped_tour_purpose'] == 'Ind-Discretionary')
                      & (spa_out_tours_df['JOINT_STATUS'] == 3),
                     'grouped_tour_purpose'] = 'Joint-Discretionary'

In [36]:
spa_out_tours_df['tour_mode'].value_counts()

SOV             23470
HOV2            11416
HOV3             9179
WALK             5259
SCHOOLBUS        1385
WALK-LOCAL       1265
WALK-PREMIUM      851
BIKE              829
PNR-LOCAL         249
OTHER             189
TNC-SINGLE        133
PNR-PREMIUM       125
WALK-MIXED        120
KNR-LOCAL          72
KNR-PREMIUM        52
TAXI               52
PNR-MIXED           7
KNR-MIXED           6
TNC-SHARED          5
Name: tour_mode, dtype: int64

In [37]:
spa_out_tours_df['auto_suff'].value_counts(dropna=False)

2    49046
1     4046
0     1572
Name: auto_suff, dtype: int64

#### On-Board Transit Survey

In [38]:
obs_df['grouped_tour_mode'] = pd.NA
obs_df.loc[obs_df['TOUR_MODE'] == 'Walk', 'grouped_tour_mode'] = 'WALK-TRANSIT'
obs_df.loc[obs_df['TOUR_MODE'] == 'PNR', 'grouped_tour_mode'] = 'PNR-TRANSIT'
obs_df.loc[obs_df['TOUR_MODE'] == 'KNR', 'grouped_tour_mode'] = 'KNR-TRANSIT'


In [39]:
obs_df['grouped_tour_mode'].value_counts()


WALK-TRANSIT    23745
PNR-TRANSIT      4812
KNR-TRANSIT      2048
Name: grouped_tour_mode, dtype: int64

In [40]:
obs_df['TOUR_PURPOSE'].value_counts(dropna=False)


Work                   18230
Ind-Discretionary       3865
University              3430
Ind-Maintenance         2903
School                   991
Work sub-tour            460
Joint-Discretionary      424
Joint-Maintenance        187
NaN                      115
Name: TOUR_PURPOSE, dtype: int64

In [41]:
# Tour purpose in recoded OBS dataset already matches desired grouped tour purpose

obs_df['grouped_tour_purpose'] = obs_df['TOUR_PURPOSE']


### Cleaning Modes and Purposes

In [42]:
# All schoolbus trips should be on school tours
spa_out_tours_df.loc[spa_out_tours_df['grouped_tour_mode'] == 'SCHOOLBUS', 'grouped_tour_purpose'] = 'School'
spa_out_trips_df.loc[spa_out_trips_df['grouped_linked_trip_mode'] == 'SCHOOLBUS', 'trip_purpose'] = 'School'

spa_out_trips_df['tour_includes_schoolbus'] = spa_out_trips_df.groupby(
    ['HH_ID', 'PER_ID', 'TOUR_ID'])['grouped_linked_trip_mode'].transform(lambda x: 1 if 'SCHOOLBUS' in x.values else 0)

spa_out_trips_df.loc[spa_out_trips_df['tour_includes_schoolbus'] == 1, 'grouped_tour_purpose'] = 'School'

In [43]:
# Reallocate to appropriate cells

spa_out_trips_df['grouped_tour_mode_updated'] = np.nan



# Drive trips on bike tours should be drive tours
spa_out_trips_df.loc[(spa_out_trips_df['grouped_tour_mode'] == 'BIKE') & \
                     (spa_out_trips_df['grouped_linked_trip_mode'].isin(['SOV', 'HOV2', 'HOV3'])),
                            'grouped_tour_mode_updated'] = spa_out_trips_df['grouped_linked_trip_mode'] 
# Bike trips only on bike tour
spa_out_trips_df.loc[(spa_out_trips_df['grouped_linked_trip_mode'] == 'BIKE'), 'grouped_tour_mode_updated'] = spa_out_trips_df['grouped_linked_trip_mode'] 

# Drive alone should be on drive tours only
spa_out_trips_df.loc[(spa_out_trips_df['grouped_linked_trip_mode'].isin(['DRIVEALONE'])) &
                      (spa_out_trips_df['grouped_tour_mode'].isin(['TNC-SINGLE', 'TNC-SHARED', 'TAXI', 'SCHOOLBUS'])),
                            'grouped_tour_mode_updated'] = spa_out_trips_df['grouped_linked_trip_mode'] 

# Drive on transit should be moved to drive  tours 
spa_out_trips_df.loc[(spa_out_trips_df['grouped_tour_mode'].isin(['WALK-TRANSIT','PNR-TRANSIT','KNR-TRANSIT']) & \
                     (spa_out_trips_df['grouped_linked_trip_mode'].isin(['SOV', 'HOV2', 'HOV3']))),
                            'grouped_tour_mode_updated'] = spa_out_trips_df['grouped_linked_trip_mode'] 

# Transit on school bus should be transit 
spa_out_trips_df.loc[(spa_out_trips_df['grouped_linked_trip_mode'].isin(['WALK-TRANSIT','PNR-TRANSIT','KNR-TRANSIT']) & \
                     (spa_out_trips_df['grouped_tour_mode'].isin(['SCHOOLBUS']))),
                            'grouped_tour_mode_updated'] = spa_out_trips_df['grouped_linked_trip_mode'] 

# KNR on TNR should be on KNR tours 
spa_out_trips_df.loc[(spa_out_trips_df['grouped_linked_trip_mode'] == 'KNR-TRANSIT')  & \
                    (spa_out_trips_df['grouped_tour_mode'] == 'PNR-TRANSIT') , 'grouped_tour_mode_updated'] = 'KNR-TRANSIT'

                     # Walk on TNR/ KNR/PNR should be on walk transit tours 
spa_out_trips_df.loc[(spa_out_trips_df['grouped_linked_trip_mode'] == 'WALK')  & \
                       ((spa_out_trips_df['grouped_tour_mode'] == 'KNR-TRANSIT') | \
                        (spa_out_trips_df['grouped_tour_mode'] == 'PNR-TRANSIT')) , 'grouped_tour_mode_updated'] = 'WALK-TRANSIT'



# Walk on TNR/ KNR/PNR should be on walk transit tours 
spa_out_trips_df.loc[(spa_out_trips_df['grouped_linked_trip_mode'] == 'WALK-TRANSIT')  & \
                       ((spa_out_trips_df['grouped_tour_mode'] == 'KNR-TRANSIT') | \
                        (spa_out_trips_df['grouped_tour_mode'] == 'PNR-TRANSIT')) , 'grouped_tour_mode_updated'] = 'WALK-TRANSIT'
# Walk trips on bike tour should be walk tour
spa_out_trips_df.loc[(spa_out_trips_df['grouped_linked_trip_mode'] == 'WALK')  & \
                    (spa_out_trips_df['grouped_tour_mode'] == 'BIKE') , 'grouped_tour_mode_updated'] = 'WALK'

       

In [44]:
# Recode tour mode

spa_out_trips_df.loc[(pd.notnull(spa_out_trips_df['grouped_tour_mode_updated'])), 'grouped_tour_mode'] = spa_out_trips_df['grouped_tour_mode_updated'] 

# Since the new tour modes do not really change the overall tour calibration targets, leaving those as-is.
# This means the data aren't 100% consistent between trips/tours when it comes to tour mode
# Updating the tour mode in the tours table (and then carrying that over to other trips on the tour) might cause more problems with working out which trip modes should be on which tour modes
#    so, best to avoid since it doesn't change the tour targets significantly.

In [45]:
display(spa_out_trips_df[(pd.notnull(spa_out_trips_df['grouped_tour_mode_updated']))])

,HH_ID,PER_ID,TOUR_ID,TRIP_ID,ORIG_PLACENO,ORIG_X,ORIG_Y,ORIG_TAZ,ORIG_MAZ,DEST_PLACENO,DEST_X,DEST_Y,DEST_TAZ,DEST_MAZ,ORIG_PURP,DEST_PURP,ORIG_ARR_HR,ORIG_ARR_MIN,ORIG_ARR_BIN,ORIG_DEP_HR,ORIG_DEP_MIN,ORIG_DEP_BIN,DEST_ARR_HR,DEST_ARR_MIN,DEST_ARR_BIN,DEST_DEP_HR,DEST_DEP_MIN,DEST_DEP_BIN,TRIP_DUR_HR,TRIP_DUR_MIN,TRIP_DUR_BIN,TRIPMODE,ISDRIVER,CHAUFFUER_ID,AUTO_OCC,TOURMODE,TOURPURP,BOARDING_PLACENO,BOARDING_PNAME,BOARDING_X,BOARDING_Y,BOARDING_TAP,ALIGHTING_PLACENO,ALIGHTING_PNAME,ALIGHTING_X,ALIGHTING_Y,ALIGHTING_TAP,TRANSIT_NUM_XFERS,TRANSIT_ROUTE_1,TRANSIT_MODE_1,XFER_1_PLACENO,XFER_1_PNAME,XFER_1_X,XFER_1_Y,XFER_1_TAP,TRANSIT_ROUTE_2,TRANSIT_MODE_2,XFER_2_PLACENO,XFER_2_PNAME,XFER_2_X,XFER_2_Y,XFER_2_TAP,TRANSIT_ROUTE_3,TRANSIT_MODE_3,XFER_3_PLACENO,XFER_3_PNAME,XFER_3_X,XFER_3_Y,XFER_3_TAP,PARKING_PLACENO,PARKING_PNAME,PARKING_X,PARKING_Y,SUBTOUR,IS_INBOUND,TRIPS_ON_JOURNEY,TRIPS_ON_TOUR,ORIG_IS_TOUR_ORIG,ORIG_IS_TOUR_DEST,DEST_IS_TOUR_DEST,DEST_IS_TOUR_ORIG,PEREXPFACT,HHEXPFACT,PERSONTYPE,FULLY_JOINT,PARTIAL_TOUR,JTRIP_ID,ESCORTED,ESCORTING,NUM_PERSONS_ESCORTED,ESCORT_PERS_1,ESCORT_PERS_2,ESCORT_PERS_3,ESCORT_PERS_4,ESCORT_PERS_5,DEST_ESCORTING,JOINT,NUM_UL_JTRIPS,DIST,ERROR,DOW,SAMPN_x,TRIPNO,PERNO,TRIP_WEIGHT,SAMPN_y,auto_suff,HH_WEIGHT_x,PER_WEIGHT,JOINT_STATUS,IS_SUBTOUR,HH_WEIGHT_y,unique_joint_trip,spa_tour_purpose,tour_purpose,grouped_tour_purpose,linked_trip_mode,tour_mode,grouped_tour_mode,grouped_linked_trip_mode,trip_purpose,tour_includes_schoolbus,grouped_tour_mode_updated
51,18116453,2,1,1,1,-93.31110,45.01565,1195,NaN,2,-93.22521,44.97439,1280,NaN,0,6,3,0,1,7,45,10,8,25,11,17,11,29,0,40,2,5,0,NaN,1,5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,2,1,0,1,0,NaN,NaN,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,5.07902,NaN,1,18116453,1,2,8.588909,18116453,2,8.588909,8.588909,1,0,8.588909,NaN,MAINTENANCE,Maintenance,Ind-Maintenance,BIKE,BIKE,BIKE,BIKE,NaN,0,BIKE
52,18116453,2,1,2,2,-93.22521,44.97439,1280,NaN,3,-93.31122,45.01559,1195,NaN,6,0,8,25,11,17,11,29,17,53,30,2,59,48,0,42,2,5,0,NaN,1,5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,2,0,1,0,1,NaN,NaN,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,6.69685,NaN,1,18116453,2,2,8.588909,18116453,2,8.588909,8.588909,1,0,8.588909,NaN,MAINTENANCE,Maintenance,Ind-Maintenance,BIKE,BIKE,BIKE,BIKE,NaN,0,BIKE
70,18118923,2,1,1,1,-93.51948,44.77350,2175,NaN,2,-93.22619,44.87441,1524,NaN,0,6,3,0,1,10,35,16,10,56,16,10,56,16,0,21,1,1,1,NaN,1,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2,5,1,0,0,0,NaN,NaN,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,20.47165,NaN,1,18118923,1,2,140.712142,18118923,2,140.712142,140.712142,1,0,140.712142,NaN,WORK,Work,Work,SOV,WALK-PREMIUM,SOV,SOV,NaN,0,SOV
73,18118923,2,1,4,4,-93.22517,44.87466,1524,NaN,5,-93.50344,44.78036,2129,NaN,6,7,22,13,39,22,21,39,22,49,40,23,2,41,0,28,2,1,1,NaN,1,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,3,5,0,0,0,0,NaN,NaN,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,19.70378,NaN,1,18118923,4,2,140.712142,18118923,2,140.712142,140.712142,1,0,140.712142,NaN,WORK,Work,Work,SOV,WALK-PREMIUM,SOV,SOV,NaN,0,SOV
74,18118923,2,1,5,5,-93.50344,44.78036,2129,NaN,6,-93.51942,44.77348,2175,NaN,7,0,22,49,40,23,2,41,23,9,41,2,59,48,0,7,1,1,1,NaN,1,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,3,5,0,0,0,1,NaN,NaN,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1.30983,NaN,1,18118923,5,2,140.712142,18118923,2,140.712142,140.712142,1,0,140.712142,NaN,WORK,Work,Work,SOV,WALK-PREMIUM,SOV,SOV,NaN

In [46]:
def remove_other_from_mode(df, mode, data_source):
    print("removing ", len(df[df[mode] == 'OTHER']), data_source, mode, "entries with OTHER mode")
    return df[df[mode] != 'OTHER']

# Remove OTHER trip and tour modes since they are not included in the model
spa_out_tours_df = remove_other_from_mode(spa_out_tours_df, mode='grouped_tour_mode', data_source='HTS')
spa_out_trips_df = remove_other_from_mode(spa_out_trips_df, mode='grouped_linked_trip_mode', data_source='HTS')
spa_out_trips_df = remove_other_from_mode(spa_out_trips_df, mode='grouped_tour_mode', data_source='HTS')


removing  189 HTS grouped_tour_mode entries with OTHER mode
removing  1642 HTS grouped_linked_trip_mode entries with OTHER mode
removing  0 HTS grouped_tour_mode entries with OTHER mode


In [47]:
spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'SCHOOLBUS']['grouped_tour_purpose'].value_counts()


School    2514
Name: grouped_tour_purpose, dtype: int64

## Trip Mode Choice

In [48]:
spa_out_trips_df.grouped_tour_mode.value_counts()


SOV             67445
HOV2            36462
HOV3            29853
WALK            11101
WALK-TRANSIT     6398
SCHOOLBUS        3037
BIKE             2311
PNR-TRANSIT       534
TNC-SINGLE        286
KNR-TRANSIT       171
TAXI              145
TNC-SHARED         15
Name: grouped_tour_mode, dtype: int64

In [49]:
spa_out_tour_trips_ct = pd.crosstab(
    spa_out_trips_df['grouped_linked_trip_mode'],
    spa_out_trips_df['grouped_tour_mode'],
    values=spa_out_trips_df['PER_WEIGHT'],
    aggfunc=sum,
    margins=True,
)

spa_out_tour_trips_ct = spa_out_tour_trips_ct.reindex(
    index=output_calibration_modes, columns=output_calibration_modes, fill_value=0)
round(spa_out_tour_trips_ct).fillna('-')

grouped_tour_mode,SOV,HOV2,HOV3,WALK,BIKE,WALK-TRANSIT,PNR-TRANSIT,KNR-TRANSIT,TAXI,TNC-SINGLE,TNC-SHARED,SCHOOLBUS,All
grouped_linked_trip_mode,,,,,,,,,,,,,
SOV,5.62875e+06,899390,414650,-,-,-,-,-,-,-,-,2714,6945506.0
HOV2,-,2.49567e+06,473535,-,-,-,-,-,-,-,-,19520,2988726.0
HOV3,-,-,2.10476e+06,-,-,-,-,-,-,-,-,29252,2134014.0
WALK,142472,104823,65067,748834,-,96733,-,-,6935,4651,113,5704,1175333.0
BIKE,-,-,-,-,135969,-,-,-,-,-,-,-,135969.0
WALK-TRANSIT,-,-,-,-,-,357440,-,-,-,-,-,-,357440.0
PNR-TRANSIT,-,-,-,-,-,-,35600,-,-,-,-,-,35600.0
KNR-TRANSIT,-,-,-,-,-,-,-,9970,-,-,-,-,9970.0
TAXI,897,43,102,-,-,1766,171,-,9395,61,-,-,12434.0


Activity based modeling data is already in this format:

#### Writing these to trip mode choice excel sheet

In [50]:
def write_tables_to_excel(dfs, excel_writer, excel_sheet_name, start_row, start_col, title, sep_for_col_title, col_title):
    # have to write first table to initialize sheet before writing title
    dfs[0].to_excel(excel_writer, excel_sheet_name, startrow=start_row+2, startcol=start_col)
    worksheet = excel_writer.sheets[excel_sheet_name]
    
    # writing title at and first table name
    worksheet.write(start_row, start_col, title)
    worksheet.write(start_row+1, start_col, dfs[0].name)
    worksheet.write(start_row+1, start_col + sep_for_col_title, col_title)
    start_row += len(dfs[0]) + 6
    
    for df in dfs[1:]:
        df.to_excel(excel_writer, excel_sheet_name, startrow=start_row, startcol=start_col)
        worksheet.write(start_row-1, start_col, df.name)
        worksheet.write(start_row-1, start_col + sep_for_col_title, col_title)
        start_row += len(df) + 4

In [51]:
def create_hts_trip_mode_choice_tables(spa_out_trips_df, purposes):
    dfs = []
    all_purposes_ct = pd.crosstab(
        spa_out_trips_df['grouped_linked_trip_mode'],
        spa_out_trips_df['grouped_tour_mode'],
        values=spa_out_trips_df['PER_WEIGHT'],
        aggfunc='sum',
        margins=True,
        dropna=False
    )
    all_purposes_ct = all_purposes_ct.reindex(
            index=output_calibration_modes, columns=output_calibration_modes, fill_value=0)
    all_purposes_ct = round(all_purposes_ct.fillna(0),0)
    
    all_purposes_ct.name = 'Purpose: Total'
    dfs.append(all_purposes_ct)
    
    # Create crosstab for each purpose
    for tour_purpose in purposes:
        trip_tour_mode_by_purp_ct = pd.crosstab(
            spa_out_trips_df[(spa_out_trips_df['grouped_tour_purpose'] == tour_purpose)]['grouped_linked_trip_mode'],
            spa_out_trips_df[(spa_out_trips_df['grouped_tour_purpose'] == tour_purpose)]['grouped_tour_mode'],
            values=spa_out_trips_df[(spa_out_trips_df['grouped_tour_purpose'] == tour_purpose)]['PER_WEIGHT'],
            aggfunc='sum',
            margins=True,
            dropna=False
        )
        trip_tour_mode_by_purp_ct = trip_tour_mode_by_purp_ct.reindex(
            index=output_calibration_modes, columns=output_calibration_modes, fill_value=0)
        trip_tour_mode_by_purp_ct = round(trip_tour_mode_by_purp_ct.fillna(0),0)
        
        trip_tour_mode_by_purp_ct.name = 'Purpose: ' + tour_purpose
        dfs.append(trip_tour_mode_by_purp_ct)
        
    return dfs


In [52]:
spa_out_trips_df['grouped_tour_purpose'].value_counts()


Ind-Maintenance        63393
Ind-Discretionary      35023
Work                   34691
School                  8344
Work sub-tour           6003
Joint-Maintenance       5005
Joint-Discretionary     4528
University               771
Name: grouped_tour_purpose, dtype: int64

In [53]:
print(output_calibration_purposes[:-1])
spa_out_trips_df['grouped_tour_mode'].value_counts()
spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'WALK-TRANSIT']['PER_WEIGHT'].sum()


['Work', 'University', 'School', 'Work sub-tour', 'Ind-Maintenance', 'Ind-Discretionary', 'Joint-Maintenance', 'Joint-Discretionary']


357439.5927666577

In [54]:
hts_trip_mc_dfs = create_hts_trip_mode_choice_tables(spa_out_trips_df, output_calibration_purposes[:-1])

## Tour Mode Choice

In [55]:
walk_tours = spa_out_tours_df[spa_out_tours_df['grouped_tour_mode'] == 'WALK-TRANSIT']['PER_WEIGHT'].sum()
walk_trips = spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'WALK-TRANSIT']['PER_WEIGHT'].sum()
walk_trips_per_tour = walk_trips / walk_tours

walk_transit_trips = spa_out_trips_df[(spa_out_trips_df['grouped_linked_trip_mode'] == 'WALK-TRANSIT') 
                                  & (spa_out_trips_df['TRIPMODE'].isin(range(6, 14 + 1)))]['PER_WEIGHT'].sum()

walk_transit_trips_per_tour = walk_transit_trips / walk_tours

pnr_tours = spa_out_tours_df[spa_out_tours_df['grouped_tour_mode'] == 'PNR-TRANSIT']['PER_WEIGHT'].sum()
pnr_trips = spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'PNR-TRANSIT']['PER_WEIGHT'].sum()
pnr_trips_per_tour = pnr_trips / pnr_tours
pnr_transit_trips = spa_out_trips_df[(spa_out_trips_df['grouped_linked_trip_mode'] == 'PNR-TRANSIT') 
                                  & (spa_out_trips_df['TRIPMODE'].isin(range(6, 14 + 1)))]['PER_WEIGHT'].sum()

pnr_transit_trips_per_tour = pnr_transit_trips / pnr_tours

knr_tours = spa_out_tours_df[spa_out_tours_df['grouped_tour_mode'] == 'KNR-TRANSIT']['PER_WEIGHT'].sum()
knr_trips = spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'KNR-TRANSIT']['PER_WEIGHT'].sum()
knr_trips_per_tour = knr_trips / knr_tours

knr_transit_trips = spa_out_trips_df[(spa_out_trips_df['grouped_linked_trip_mode'] == 'KNR-TRANSIT') 
                                  & (spa_out_trips_df['TRIPMODE'].isin(range(6, 14 + 1)))]['PER_WEIGHT'].sum()

knr_transit_trips_per_tour = knr_transit_trips / knr_tours



print("Trips Per Tour for ")
print("WALK-TRANSIT Tours:", walk_tours, "\t Linked Trips:", walk_trips,
      "\t Trips/Tour:",  round(walk_trips_per_tour, 3), "\t Transit Trips/Tour: ", round(walk_transit_trips_per_tour,3))
print("PNR-TRANSIT Tours:", pnr_tours, "\t Linked Trips:", pnr_trips,
      "\t Trips/Tour:",  round(pnr_trips_per_tour, 3), "\t Transit Trips/Tour: ", round(pnr_transit_trips_per_tour,3))
print("KNR-TRANSIT Tours:", knr_tours, "\t Linked Trips:", knr_trips,
      "\t Trips/Tour:",  round(knr_trips_per_tour, 3), "\t Transit Trips/Tour: ", round(knr_transit_trips_per_tour,3))

# tnr and knr are the same; but it looks like that's correct

Trips Per Tour for 
WALK-TRANSIT Tours: 167414.68405147106 	 Linked Trips: 357439.5927666577 	 Trips/Tour: 2.135 	 Transit Trips/Tour:  2.135
PNR-TRANSIT Tours: 25750.91419542684 	 Linked Trips: 35599.68710314633 	 Trips/Tour: 1.382 	 Transit Trips/Tour:  1.382
KNR-TRANSIT Tours: 8252.638334719515 	 Linked Trips: 9969.64099466292 	 Trips/Tour: 1.208 	 Transit Trips/Tour:  1.208


In [56]:
def get_transit_trips_per_tour(df, grouped_tour_mode):
    tours = df[df['grouped_tour_mode'] == grouped_tour_mode]['PER_WEIGHT'].sum()
    transit_trips = df[(df['grouped_tour_mode'] == goruped_tour_mode)
                      & (df['TRIPMODE'].isin(range(9, 17 + 1)))]['PER_WEIGHT'].sum()
    return transit_trips/tours

transit_trips_per_transit_tour_df = pd.DataFrame(index=output_calibration_modes)
transit_trips_per_transit_tour_df['transit_trips_per_transit_tour'] = pd.NA
transit_trips_per_transit_tour_df.loc['WALK-TRANSIT', 'transit_trips_per_transit_tour'] = walk_transit_trips_per_tour
transit_trips_per_transit_tour_df.loc['PNR-TRANSIT', 'transit_trips_per_transit_tour'] = pnr_transit_trips_per_tour
transit_trips_per_transit_tour_df.loc['KNR-TRANSIT', 'transit_trips_per_transit_tour'] = knr_transit_trips_per_tour
transit_trips_per_transit_tour_df.name = 'Purpose: Total'
transit_trips_per_transit_tour_df

# Not high enough -- replace pnr/knr with 2.0
transit_trips_per_transit_tour_df.loc['PNR-TRANSIT', 'transit_trips_per_transit_tour'] = 2.0
transit_trips_per_transit_tour_df.loc['KNR-TRANSIT', 'transit_trips_per_transit_tour'] = 2.0


In [57]:
transit_trips_per_transit_tour_df

,transit_trips_per_transit_tour
SOV,<NA>
HOV2,<NA>
HOV3,<NA>
WALK,<NA>
BIKE,<NA>
WALK-TRANSIT,2.13506
PNR-TRANSIT,2
KNR-TRANSIT,2
TAXI,<NA>
TNC-SINGLE,<NA>


In [58]:
obs_df['tourweight'] = obs_df['linked_weight_scaled']
obs_df.loc[obs_df['grouped_tour_mode'] == 'WALK-TRANSIT','tourweight'] =  obs_df.loc[
    obs_df['grouped_tour_mode'] == 'WALK-TRANSIT', 'tourweight'] / transit_trips_per_transit_tour_df.loc[
    'WALK-TRANSIT', 'transit_trips_per_transit_tour']
obs_df.loc[obs_df['grouped_tour_mode'] == 'PNR-TRANSIT', 'tourweight'] =  obs_df.loc[
    obs_df['grouped_tour_mode'] == 'PNR-TRANSIT', 'tourweight'] / transit_trips_per_transit_tour_df.loc[
    'PNR-TRANSIT', 'transit_trips_per_transit_tour']
obs_df.loc[obs_df['grouped_tour_mode'] == 'KNR-TRANSIT', 'tourweight'] =  obs_df.loc[
    obs_df['grouped_tour_mode'] == 'KNR-TRANSIT', 'tourweight'] / transit_trips_per_transit_tour_df.loc[
    'KNR-TRANSIT', 'transit_trips_per_transit_tour']

print("Total number of On-board survey Tours: ", int(obs_df['tourweight'].sum()))
print("Total number of On-board survey Trips: ", int(obs_df['linked_weight_scaled'].sum()))

Total number of On-board survey Tours:  129191
Total number of On-board survey Trips:  272362


In [59]:
obs_df.grouped_tour_mode.value_counts()

WALK-TRANSIT    23745
PNR-TRANSIT      4812
KNR-TRANSIT      2048
Name: grouped_tour_mode, dtype: int64

In [60]:
def create_hts_tour_mode_choice_tables(spa_out_tours_df, purposes):
    dfs = []
    all_purposes_ct = pd.crosstab(
        spa_out_tours_df['grouped_tour_mode'],
        spa_out_tours_df['auto_suff'],
        values=spa_out_tours_df['PER_WEIGHT'],
        aggfunc='sum',
        margins=True,
        dropna=False
    )
    all_purposes_ct = all_purposes_ct.reindex(
            index=output_calibration_modes, fill_value=0)
    all_purposes_ct = round(all_purposes_ct.fillna(0),0)
    
    all_purposes_ct.name = 'Purpose: Total'
    dfs.append(all_purposes_ct)
    
    # Create crosstab for each purpose
    for tour_purpose in purposes:
        tour_mode_autosuff_by_purp_ct = pd.crosstab(
            spa_out_tours_df[(spa_out_tours_df['grouped_tour_purpose'] == tour_purpose)]['grouped_tour_mode'],
            spa_out_tours_df[(spa_out_tours_df['grouped_tour_purpose'] == tour_purpose)]['auto_suff'],
            values=spa_out_tours_df[(spa_out_tours_df['grouped_tour_purpose'] == tour_purpose)]['PER_WEIGHT'],
            aggfunc='sum',
            margins=True,
            dropna=False
        )
        tour_mode_autosuff_by_purp_ct = tour_mode_autosuff_by_purp_ct.reindex(
            index=output_calibration_modes, fill_value=0)
        tour_mode_autosuff_by_purp_ct = round(tour_mode_autosuff_by_purp_ct.fillna(0),0)
        
        tour_mode_autosuff_by_purp_ct.name = 'Purpose: ' + tour_purpose
        dfs.append(tour_mode_autosuff_by_purp_ct)
        
    return dfs


def create_obs_tour_mode_choice_tables(obs_df, purposes, weight):
    dfs = []
    all_purposes_ct = pd.crosstab(
        obs_df['grouped_tour_mode'],
        obs_df['auto_suff'],
        values=obs_df[weight],
        aggfunc='sum',
        margins=True,
        dropna=False
    )
    all_purposes_ct = all_purposes_ct.reindex(
            index=output_calibration_modes, fill_value=0)
    all_purposes_ct = round(all_purposes_ct.fillna(0),0)
    
    all_purposes_ct.name = 'Purpose: Total'
    dfs.append(all_purposes_ct)
    
    # Create crosstab for each purpose
    for tour_purpose in purposes:
        tour_mode_autosuff_by_purp_ct = pd.crosstab(
            obs_df[(obs_df['grouped_tour_purpose'] == tour_purpose)]['grouped_tour_mode'],
            obs_df[(obs_df['grouped_tour_purpose'] == tour_purpose)]['auto_suff'],
            values=obs_df[weight],
            aggfunc='sum',
            margins=True,
            dropna=False
        )
        tour_mode_autosuff_by_purp_ct = tour_mode_autosuff_by_purp_ct.reindex(
            index=output_calibration_modes, fill_value=0)
        tour_mode_autosuff_by_purp_ct = round(tour_mode_autosuff_by_purp_ct.fillna(0),0)
        
        tour_mode_autosuff_by_purp_ct.name = 'Purpose: ' + tour_purpose
        dfs.append(tour_mode_autosuff_by_purp_ct)
        
    return dfs

In [61]:
def split_obs_ind_to_joint_plus_ind(obs_tour_mc_dfs, hts_transit_df):
    joint_disc_tours = hts_transit_df[hts_transit_df['grouped_tour_purpose']=='Joint-Discretionary']['PER_WEIGHT'].sum()
    ind_disc_tours = hts_transit_df[hts_transit_df['grouped_tour_purpose']=='Ind-Discretionary']['PER_WEIGHT'].sum()
    percent_ind_disc_tours = ind_disc_tours / (joint_disc_tours + ind_disc_tours)
    
    joint_maint_tours = hts_transit_df[hts_transit_df['grouped_tour_purpose']=='Joint-Maintenance']['PER_WEIGHT'].sum()
    ind_maint_tours = hts_transit_df[hts_transit_df['grouped_tour_purpose']=='Ind-Maintenance']['PER_WEIGHT'].sum()
    percent_ind_maint_tours = ind_maint_tours / (joint_maint_tours + ind_maint_tours)
    
    print("Household Travel Survey has")
    print("Maintenance: ", round(percent_ind_disc_tours*100,2), '% Individual, ', round((1-percent_ind_disc_tours)*100,2), '% Joint')
    print("Discretionary: ", round(percent_ind_maint_tours*100,2), '% Individual, ', round((1-percent_ind_maint_tours)*100,2), '% Joint')

    output_dfs = []
    ind_maint_df = pd.DataFrame()
    ind_disc_df = pd.DataFrame()
    joint_maint_df = pd.DataFrame()
    joint_disc_df = pd.DataFrame()
    
    for df in obs_tour_mc_dfs:
        if 'Ind-Maintenance' in df.name:
            ind_maint_df = round(df * percent_ind_maint_tours)
            ind_maint_df.name = 'Purpose: Ind-Maintenance'
            joint_maint_df = round(df * (1 - percent_ind_maint_tours))
            joint_maint_df.name = 'Purpose: Joint-Maintenance'
        elif 'Ind-Discretionary' in df.name:
            ind_disc_df = round(df * percent_ind_disc_tours)
            ind_disc_df.name = 'Purpose: Ind-Discretionary'
            joint_disc_df = round(df * (1 - percent_ind_disc_tours))
            joint_disc_df.name = 'Purpose: Joint-Discretionary'
        else:
            output_dfs.append(df)
            
    output_dfs.append(ind_maint_df)
    output_dfs.append(ind_disc_df)
    output_dfs.append(joint_maint_df)
    output_dfs.append(joint_disc_df)
    
    total_tours_before_split = sum([df.sum().sum() for df in obs_tour_mc_dfs])
    total_tours_after_split = sum([df.sum().sum() for df in output_dfs])
    assert abs(total_tours_before_split - total_tours_after_split) < 1, "Tours not conserved!!!"
            
    return output_dfs

In [62]:
hts_tour_mc_dfs = create_hts_tour_mode_choice_tables(spa_out_tours_df, output_calibration_purposes[:-1])
obs_tour_mc_trips_dfs = create_obs_tour_mode_choice_tables(obs_df, output_calibration_purposes[:-1], weight='linked_weight_scaled')
obs_tour_mc_tours_dfs = create_obs_tour_mode_choice_tables(obs_df, output_calibration_purposes[:-1], weight='tourweight')

# splitting obs maintenance and discretionary tour purposes into joint and ind based on hts split
# hts_transit_trips_df = spa_out_trips_df[spa_out_trips_df['grouped_tour_mode'].isin(['WALK-TRANSIT','PNR-TRANSIT', 'KNR-TRANSIT'])]
# obs_tour_mc_trips_dfs = split_obs_ind_to_joint_plus_ind(obs_tour_mc_trips_dfs, hts_transit_trips_df)

# hts_transit_tours_df = spa_out_tours_df[spa_out_tours_df['grouped_tour_mode'].isin(['WALK-TRANSIT','PNR-TRANSIT', 'KNR-TRANSIT'])]
# obs_tour_mc_tours_dfs = split_obs_ind_to_joint_plus_ind(obs_tour_mc_tours_dfs, hts_transit_tours_df)

In [63]:
obs_tour_mc_tours_dfs

[auto_suff                0        1        2       All
 grouped_tour_mode                                     
 SOV                    0.0      0.0      0.0       0.0
 HOV2                   0.0      0.0      0.0       0.0
 HOV3                   0.0      0.0      0.0       0.0
 WALK                   0.0      0.0      0.0       0.0
 BIKE                   0.0      0.0      0.0       0.0
 WALK-TRANSIT       42997.0  29436.0  31074.0  103507.0
 PNR-TRANSIT          418.0   2440.0  15071.0   17929.0
 KNR-TRANSIT         2037.0   2811.0   2908.0    7755.0
 TAXI                   0.0      0.0      0.0       0.0
 TNC-SINGLE             0.0      0.0      0.0       0.0
 TNC-SHARED             0.0      0.0      0.0       0.0
 SCHOOLBUS              0.0      0.0      0.0       0.0
 All                45451.0  34688.0  49052.0  129192.0,
 auto_suff                0        1        2      All
 grouped_tour_mode                                    
 SOV                    0.0      0.0      0.0    

In [64]:
def create_final_tour_mc_targets(obs_tour_mc_tours_dfs, hts_tour_mc_dfs):
    tour_mc_calib_targets_dfs = []
    tour_mc_scaling_factors_dfs = []
    tour_mc_calib_targets_pct_dfs = []
    for i in range(len(obs_tour_mc_tours_dfs)):
        obs_tours_df = obs_tour_mc_tours_dfs[i]
        hts_tours_df = hts_tour_mc_dfs[i]
        assert obs_tours_df.name == hts_tours_df.name, "Purposes not the same!"
        
        # building calibration targets
        calibration_targets_df = hts_tours_df.copy()
        calibration_targets_df.name = hts_tours_df.name
        
        calibration_targets_df.loc[calibration_targets_df.index.isin(
            ['WALK-TRANSIT','PNR-TRANSIT', 'KNR-TRANSIT'])] = obs_tours_df.loc[obs_tours_df.index.isin(
            ['WALK-TRANSIT','PNR-TRANSIT', 'KNR-TRANSIT'])]
        calibration_targets_df.loc['All'] = calibration_targets_df.drop(labels='All', axis=0, inplace=False).sum()
        tour_mc_calib_targets_dfs.append(calibration_targets_df)
        
        # building scaling factors
        scaling_factors_df = pd.DataFrame(index=output_calibration_modes)
        scaling_factors_df['scaling_factor'] = pd.NA
        scaling_factors_df.loc['WALK-TRANSIT', 'scaling_factor'] = calibration_targets_df.loc['WALK-TRANSIT', 'All'] / hts_tours_df.loc['WALK-TRANSIT', 'All']
        scaling_factors_df.loc['PNR-TRANSIT', 'scaling_factor'] = calibration_targets_df.loc['PNR-TRANSIT', 'All'] / hts_tours_df.loc['PNR-TRANSIT', 'All']
        scaling_factors_df.loc['KNR-TRANSIT', 'scaling_factor'] = calibration_targets_df.loc['KNR-TRANSIT', 'All'] / hts_tours_df.loc['KNR-TRANSIT', 'All']
        scaling_factors_df.replace(to_replace=np.inf, value=0, inplace=True)
        scaling_factors_df.name = calibration_targets_df.name
        tour_mc_scaling_factors_dfs.append(scaling_factors_df)
        
        # create distribution table
        calibration_targets_pct_df = calibration_targets_df.copy()
        calibration_targets_pct_df.drop(labels='All', axis=0, inplace=True)
        cols = list(calibration_targets_pct_df.columns)
        calibration_targets_pct_df[cols] = calibration_targets_pct_df[cols].div(calibration_targets_pct_df[cols].sum(axis=0), axis=1)

        #trip_mc_calib_targets_pct_df.loc['All'] = trip_mc_calib_targets_pct_df.sum()
        calibration_targets_pct_df.loc['All'] = calibration_targets_pct_df.sum(axis=0)
        
        calibration_targets_pct_df.name = calibration_targets_df.name
        
        tour_mc_calib_targets_pct_dfs.append(calibration_targets_pct_df)
        
    return tour_mc_calib_targets_dfs, tour_mc_scaling_factors_dfs, tour_mc_calib_targets_pct_dfs
        


In [65]:
tour_mc_calib_targets_dfs, tour_mc_scaling_factors_dfs, tour_mc_calib_targets_pct_dfs = create_final_tour_mc_targets(obs_tour_mc_tours_dfs, hts_tour_mc_dfs)

C:\Users\leah.flake\.conda\envs\mcasim\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\leah.flake\.conda\envs\mcasim\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in double_scalars


In [66]:
# Creating a table that is the sum of all non-Total purposes for consistency check
tour_mc_summed_purposes_df = tour_mc_calib_targets_dfs[1].copy()
for i in range(2, len(tour_mc_calib_targets_dfs)):
    tour_mc_summed_purposes_df = tour_mc_summed_purposes_df + tour_mc_calib_targets_dfs[i]
tour_mc_summed_purposes_df.name = 'Non-Total Sum'
tour_mc_calib_targets_dfs.append(tour_mc_summed_purposes_df)

In [67]:
tour_mc_scaling_factors_dfs

[             scaling_factor
 SOV                    <NA>
 HOV2                   <NA>
 HOV3                   <NA>
 WALK                   <NA>
 BIKE                   <NA>
 WALK-TRANSIT       0.618266
 PNR-TRANSIT        0.696245
 KNR-TRANSIT        0.939658
 TAXI                   <NA>
 TNC-SINGLE             <NA>
 TNC-SHARED             <NA>
 SCHOOLBUS              <NA>
 All                    <NA>,
              scaling_factor
 SOV                    <NA>
 HOV2                   <NA>
 HOV3                   <NA>
 WALK                   <NA>
 BIKE                   <NA>
 WALK-TRANSIT        1.10649
 PNR-TRANSIT        0.898034
 KNR-TRANSIT         4.83197
 TAXI                   <NA>
 TNC-SINGLE             <NA>
 TNC-SHARED             <NA>
 SCHOOLBUS              <NA>
 All                    <NA>,
              scaling_factor
 SOV                    <NA>
 HOV2                   <NA>
 HOV3                   <NA>
 WALK                   <NA>
 BIKE                   <NA>
 WALK-TRANSI

In [68]:
# Writing Tour Mode Choice targets
# HTS tours
write_tables_to_excel(
    dfs=hts_tour_mc_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='tour_mode_choice',
    start_row=0,
    start_col=0,
    title='HTS Survey Tours',
    sep_for_col_title=3,
    col_title='Auto Sufficiency'
)

# OBS trips
write_tables_to_excel(
    dfs=obs_tour_mc_trips_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='tour_mode_choice',
    start_row=0,
    start_col=7,
    title='On-Board Survey Trips',
    sep_for_col_title=3,
    col_title='Auto Sufficiency'
)

# Transit Trips per Transit tour
# using the same scaling factor for all trip purposes
transit_trips_per_transit_tour_dfs = [transit_trips_per_transit_tour_df for i in range(len(obs_tour_mc_trips_dfs))]
write_tables_to_excel(
    dfs=transit_trips_per_transit_tour_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='tour_mode_choice',
    start_row=0,
    start_col=13,
    title='OBS Trips to Tours Factors',
    sep_for_col_title=1,
    col_title=''
)

# OBS tours
write_tables_to_excel(
    dfs=obs_tour_mc_tours_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='tour_mode_choice',
    start_row=0,
    start_col=16,
    title='On-Board Survey Tours',
    sep_for_col_title=3,
    col_title='Auto Sufficiency'
)

# Calibration Targets
write_tables_to_excel(
    dfs=tour_mc_calib_targets_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='tour_mode_choice',
    start_row=0,
    start_col=23,
    title='Calibration Targets',
    sep_for_col_title=3,
    col_title='Auto Sufficiency'
)


# Calibration Targets distributions
write_tables_to_excel(
    dfs=tour_mc_calib_targets_pct_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='tour_mode_choice',
    start_row=0,
    start_col=30,
    title='Calibration Targets - Distribution',
    sep_for_col_title=3,
    col_title='Auto Sufficiency'
)


### Creating final Trip Mode Choice targets

In [69]:
def create_trip_mode_choice_targets(hts_trip_mc_dfs, obs_tour_mc_trips_dfs):
    trip_mc_calib_targets_dfs = []
    obs_trip_target_dfs = []
    trip_mc_scaling_factors_dfs = []
    hts_trip_mc_dfs_updated = []
    trip_mc_calib_targets_pct_dfs = []
    for i in range(len(obs_tour_mc_tours_dfs)):
        hts_trip_tour_mode_df = hts_trip_mc_dfs[i]
        obs_trips_df = obs_tour_mc_trips_dfs[i]
        assert obs_trips_df.name == hts_trip_tour_mode_df.name, "Purposes not the same!"
        
        # Copying OBS trip targets and Scaling
        obs_trip_target_df = pd.DataFrame(index=output_calibration_modes)
        obs_trip_target_df['OBS_trips'] = pd.NA
        obs_trip_target_df['row_scaling_factor'] = pd.NA
        obs_trip_target_df['col_scaling_factor'] = pd.NA
        trip_mc_calib_targets_df = hts_trip_tour_mode_df.copy()

        transit_modes = ['WALK-TRANSIT', 'PNR-TRANSIT', 'KNR-TRANSIT']
        
        # display(hts_trip_tour_mode_df)
        for transit_mode in transit_modes:

     
            # final trip targets come from OBS
            obs_trip_target_df.loc[transit_mode, 'OBS_trips'] = obs_trips_df.loc[transit_mode, 'All']
            # scaling linked-trips to match trip targets from OBS
            obs_trip_target_df.loc[transit_mode, 'row_scaling_factor'] = \
                obs_trip_target_df.loc[transit_mode, 'OBS_trips'] / hts_trip_tour_mode_df.loc[transit_mode, 'All']
            trip_mc_calib_targets_df.loc[transit_mode] = \
                trip_mc_calib_targets_df.loc[transit_mode] * obs_trip_target_df.loc[transit_mode, 'row_scaling_factor']

            # need to scale such that the trip distribution between non-transit and transit trips 
            #  for the transit mode remain the same

            pre_scaled_transit_tours = hts_trip_tour_mode_df.loc[transit_modes, transit_mode].sum()
            print(pre_scaled_transit_tours)
            scaled_transit_tours = trip_mc_calib_targets_df.loc[transit_modes, transit_mode].sum()
            print(scaled_transit_tours)

            # creating scaling factor for non-transit trips in transit tours
            obs_trip_target_df.loc[transit_mode, 'col_scaling_factor'] = scaled_transit_tours / pre_scaled_transit_tours
            print(obs_trip_target_df)
            # applying that scaling factor to non-transit trips
            trip_mc_calib_targets_df.loc[~hts_trip_tour_mode_df.index.isin(transit_modes), transit_mode] = \
                    hts_trip_tour_mode_df.loc[~hts_trip_tour_mode_df.index.isin(transit_modes), transit_mode] \
                    * obs_trip_target_df.loc[transit_mode, 'col_scaling_factor']
            
            # when there are no transit trips in HTS, but there are targets in OBS survey, all OBS trips are assumed symmetric
            if (hts_trip_tour_mode_df.loc[transit_mode, transit_modes].sum() == 0):
                trip_mc_calib_targets_df.loc[transit_mode, transit_mode] = obs_trip_target_df.loc[transit_mode, 'OBS_trips']
        

        
        obs_trip_target_df.loc['All', 'OBS_trips'] = obs_trip_target_df['OBS_trips'].sum()
        obs_trip_target_df.replace(to_replace=np.inf, value=0, inplace=True)
        obs_trip_target_df.name = obs_trips_df.name
        obs_trip_target_dfs.append(obs_trip_target_df)
        

        trip_mc_calib_targets_df.fillna(0, inplace=True)

        trip_mc_calib_targets_df.loc['All'] = trip_mc_calib_targets_df.drop(labels='All', axis=0, inplace=False).sum()
        trip_mc_calib_targets_df.loc[:,'All'] = trip_mc_calib_targets_df.drop(labels='All', axis=1, inplace=False).sum(axis=1)
        trip_mc_calib_targets_df = round(trip_mc_calib_targets_df)
        trip_mc_calib_targets_df.name = obs_trip_target_df.name
        
        # create distribution table
        trip_mc_calib_targets_pct_df = trip_mc_calib_targets_df.copy()
        trip_mc_calib_targets_pct_df.drop(labels='All', axis=0, inplace=True)
        cols = list(trip_mc_calib_targets_pct_df.columns)
        trip_mc_calib_targets_pct_df[cols] = trip_mc_calib_targets_pct_df[cols].div(trip_mc_calib_targets_pct_df[cols].sum(axis=0), axis=1)

        #trip_mc_calib_targets_pct_df.loc['All'] = trip_mc_calib_targets_pct_df.sum()
        trip_mc_calib_targets_pct_df.loc['All'] = trip_mc_calib_targets_pct_df.sum(axis=0)
        
        trip_mc_calib_targets_pct_df.name = trip_mc_calib_targets_df.name
        
        trip_mc_calib_targets_dfs.append(trip_mc_calib_targets_df)
        hts_trip_mc_dfs_updated.append(hts_trip_tour_mode_df)
        trip_mc_calib_targets_pct_dfs.append(trip_mc_calib_targets_pct_df)

    return obs_trip_target_dfs, trip_mc_calib_targets_dfs, hts_trip_mc_dfs_updated, trip_mc_calib_targets_pct_dfs

In [70]:
hts_trip_mc_dfs_init = hts_trip_mc_dfs.copy()
obs_trip_target_dfs, trip_mc_calib_targets_dfs, hts_trip_mc_dfs, trip_mc_calib_targets_pct_dfs = create_trip_mode_choice_targets(
    hts_trip_mc_dfs_init, obs_tour_mc_trips_dfs)

357440.0
220994.0
             OBS_trips row_scaling_factor col_scaling_factor
SOV               <NA>               <NA>               <NA>
HOV2              <NA>               <NA>               <NA>
HOV3              <NA>               <NA>               <NA>
WALK              <NA>               <NA>               <NA>
BIKE              <NA>               <NA>               <NA>
WALK-TRANSIT    220994           0.618269           0.618269
PNR-TRANSIT       <NA>               <NA>               <NA>
KNR-TRANSIT       <NA>               <NA>               <NA>
TAXI              <NA>               <NA>               <NA>
TNC-SINGLE        <NA>               <NA>               <NA>
TNC-SHARED        <NA>               <NA>               <NA>
SCHOOLBUS         <NA>               <NA>               <NA>
All               <NA>               <NA>               <NA>
35600.0
35858.0
             OBS_trips row_scaling_factor col_scaling_factor
SOV               <NA>               <NA>          

97.0
264.0
             OBS_trips row_scaling_factor col_scaling_factor
SOV               <NA>               <NA>               <NA>
HOV2              <NA>               <NA>               <NA>
HOV3              <NA>               <NA>               <NA>
WALK              <NA>               <NA>               <NA>
BIKE              <NA>               <NA>               <NA>
WALK-TRANSIT      3317            2.16656            2.16656
PNR-TRANSIT        110                inf                NaN
KNR-TRANSIT        264            2.72165            2.72165
TAXI              <NA>               <NA>               <NA>
TNC-SINGLE        <NA>               <NA>               <NA>
TNC-SHARED        <NA>               <NA>               <NA>
SCHOOLBUS         <NA>               <NA>               <NA>
All               <NA>               <NA>               <NA>


C:\Users\leah.flake\.conda\envs\mcasim\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\leah.flake\.conda\envs\mcasim\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\leah.flake\.conda\envs\mcasim\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\leah.flake\.conda\envs\mcasim\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\leah.flake\.conda\envs\mcasim\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\leah.flake\.conda\envs\mcasim\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\leah.flake\.conda\envs\mcasim\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in double_sca

In [71]:
# Creating a table that is the sum of all non-Total purposes for consistency check
summed_purposes_df = trip_mc_calib_targets_dfs[1].copy()
for i in range(2, len(trip_mc_calib_targets_dfs)):
    summed_purposes_df = summed_purposes_df + trip_mc_calib_targets_dfs[i]
summed_purposes_df.name = 'Non-Total Sum'
trip_mc_calib_targets_dfs.append(summed_purposes_df)

In [72]:
hts_trip_mc_dfs[0]


grouped_tour_mode,SOV,HOV2,HOV3,WALK,BIKE,WALK-TRANSIT,PNR-TRANSIT,KNR-TRANSIT,TAXI,TNC-SINGLE,TNC-SHARED,SCHOOLBUS,All
grouped_linked_trip_mode,,,,,,,,,,,,,
SOV,5628751.0,899390.0,414650.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2714.0,6945506.0
HOV2,0.0,2495671.0,473535.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19520.0,2988726.0
HOV3,0.0,0.0,2104762.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29252.0,2134014.0
WALK,142472.0,104823.0,65067.0,748834.0,0.0,96733.0,0.0,0.0,6935.0,4651.0,113.0,5704.0,1175333.0
BIKE,0.0,0.0,0.0,0.0,135969.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,135969.0
WALK-TRANSIT,0.0,0.0,0.0,0.0,0.0,357440.0,0.0,0.0,0.0,0.0,0.0,0.0,357440.0
PNR-TRANSIT,0.0,0.0,0.0,0.0,0.0,0.0,35600.0,0.0,0.0,0.0,0.0,0.0,35600.0
KNR-TRANSIT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9970.0,0.0,0.0,0.0,0.0,9970.0
TAXI,897.0,43.0,102.0,0.0,0.0,1766.0,171.0,0.0,9395.0,61.0,0.0,0.0,12434.0


In [73]:
# writing HTS trip mode choice targets
write_tables_to_excel(
    dfs=hts_trip_mc_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='trip_mode_choice',
    start_row=0,
    start_col=0,
    title='HTS Survey Trips',
    sep_for_col_title=5,
    col_title='Tour Mode'
)

# OBS trips and scaling factors
write_tables_to_excel(
    dfs=obs_trip_target_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='trip_mode_choice',
    start_row=0,
    start_col=15,
    title='OBS Trip Targets',
    sep_for_col_title=1,
    col_title=''
)

# Trip Mode Targets
write_tables_to_excel(
    dfs=trip_mc_calib_targets_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='trip_mode_choice',
    start_row=0,
    start_col=20,
    title='Trip Mode Choice Targets',
    sep_for_col_title=5,
    col_title='Tour Mode'
)

# Trip Mode Targets
write_tables_to_excel(
    dfs=trip_mc_calib_targets_pct_dfs,
    excel_writer=excel_writer,
    excel_sheet_name='trip_mode_choice',
    start_row=0,
    start_col=36,
    title='Trip Mode Choice Targets - Distribution',
    sep_for_col_title=5,
    col_title='Tour Mode'
)

In [74]:
excel_writer.save()
excel_writer.close()

C:\Users\leah.flake\.conda\envs\mcasim\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


## Writing Final Tables

In [75]:
print('Tour mode choice calibration targets written to: ', tour_mode_choice_output_path)
for df in tour_mc_calib_targets_dfs:
    table_name = df.name.replace('Purpose: ','') + '.csv'
    print(table_name)
    df.to_csv(os.path.join(tour_mode_choice_output_path, table_name))

print('Trip mode choice calibration targets written to: ', trip_mode_choice_output_path)
for df in trip_mc_calib_targets_dfs:
    table_name = df.name.replace('Purpose: ','') + '.csv'
    print(table_name)
    df.to_csv(os.path.join(trip_mode_choice_output_path, table_name))

Tour mode choice calibration targets written to:  output\tour_mode_choice
Total.csv
Work.csv
University.csv
School.csv
Work sub-tour.csv
Ind-Maintenance.csv
Ind-Discretionary.csv
Joint-Maintenance.csv
Joint-Discretionary.csv
Non-Total Sum.csv
Trip mode choice calibration targets written to:  output\trip_mode_choice
Total.csv
Work.csv
University.csv
School.csv
Work sub-tour.csv
Ind-Maintenance.csv
Ind-Discretionary.csv
Joint-Maintenance.csv
Joint-Discretionary.csv
Non-Total Sum.csv


In [76]:
# Merging final Tables into one output table for the automated calibration script
def melt_df(df, melt_id_var, value_name):
    melted_df = df.reset_index().melt(id_vars=[melt_id_var])
    melted_df.rename(columns={'value': value_name}, inplace=True)
    melted_df['purpose'] = df.name.replace('Purpose: ', '')
    return melted_df

melted_dfs = []
# Don't want to include the non-total sum table
for df in tour_mc_calib_targets_dfs[:-1]:
    melted_df = melt_df(df, melt_id_var='grouped_tour_mode', value_name='tours')
    melted_dfs.append(melted_df)
tour_mode_choice_calibration_table = pd.concat(melted_dfs)

melted_dfs = []
# Don't want to include the non-total sum table
for df in trip_mc_calib_targets_dfs[:-1]:
    melted_df = melt_df(df, melt_id_var='grouped_linked_trip_mode', value_name='trips')
    melted_dfs.append(melted_df)
trip_mode_choice_calibration_table = pd.concat(melted_dfs)

tour_mode_choice_calibration_table.to_csv(os.path.join(output_path, 'tour_mode_choice_calibration_targets.csv'), index=False)
trip_mode_choice_calibration_table.to_csv(os.path.join(output_path, 'trip_mode_choice_calibration_targets.csv'), index=False)

## Output Statistics

In [77]:
trip_counts_df = pd.DataFrame(index=['WALK-TRANSIT', 'PNR-TRANSIT', 'KNR-TRANSIT'])
trip_counts_df['HTS Raw'] = pd.NA
trip_counts_df.loc['WALK-TRANSIT', 'HTS Raw'] = len(spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'WALK-TRANSIT'])
trip_counts_df.loc['PNR-TRANSIT', 'HTS Raw'] = len(spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'PNR-TRANSIT'])
trip_counts_df.loc['KNR-TRANSIT', 'HTS Raw'] = len(spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'KNR-TRANSIT'])

trip_counts_df['HTS Weighted'] = pd.NA
trip_counts_df.loc['WALK-TRANSIT', 'HTS Weighted'] = spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'WALK-TRANSIT']['PER_WEIGHT'].sum()
trip_counts_df.loc['PNR-TRANSIT', 'HTS Weighted'] = spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'PNR-TRANSIT']['PER_WEIGHT'].sum()
trip_counts_df.loc['KNR-TRANSIT', 'HTS Weighted'] = spa_out_trips_df[spa_out_trips_df['grouped_linked_trip_mode'] == 'KNR-TRANSIT']['PER_WEIGHT'].sum()

trip_counts_df['OBS Raw'] = pd.NA
trip_counts_df.loc['WALK-TRANSIT', 'OBS Raw'] = len(obs_df[obs_df['grouped_tour_mode'] == 'WALK-TRANSIT'])
trip_counts_df.loc['PNR-TRANSIT', 'OBS Raw'] = len(obs_df[obs_df['grouped_tour_mode'] == 'PNR-TRANSIT'])
trip_counts_df.loc['KNR-TRANSIT', 'OBS Raw'] = len(obs_df[obs_df['grouped_tour_mode'] == 'KNR-TRANSIT'])

trip_counts_df['OBS Weighted'] = pd.NA
trip_counts_df.loc['WALK-TRANSIT', 'OBS Weighted'] = obs_df[obs_df['grouped_tour_mode'] == 'WALK-TRANSIT']['linked_weight_scaled'].sum()
trip_counts_df.loc['PNR-TRANSIT', 'OBS Weighted'] = obs_df[obs_df['grouped_tour_mode'] == 'PNR-TRANSIT']['linked_weight_scaled'].sum()
trip_counts_df.loc['KNR-TRANSIT', 'OBS Weighted'] = obs_df[obs_df['grouped_tour_mode'] == 'KNR-TRANSIT']['linked_weight_scaled'].sum()

trip_counts_df.loc['All'] = trip_counts_df.sum()
print('Trips in each survey:')
trip_counts_df

Trips in each survey:


,HTS Raw,HTS Weighted,OBS Raw,OBS Weighted
WALK-TRANSIT,4744,357440,23745,220994
PNR-TRANSIT,528,35599.7,4812,35858.4
KNR-TRANSIT,167,9969.64,2048,15510.4
All,5439,403009,30605,272363


In [78]:

    
# creating output tables for visualizer
mode_vis_dict = {
    'SOV': 1,
    'HOV2': 2,
    'HOV3': 3,
    'WALK': 4,
    'BIKE': 5,
    'WALK-TRANSIT': 6,
    'PNR-TRANSIT': 7,
    'KNR-TRANSIT': 8,
    'SCHOOLBUS': 10,
    'TAXI': 9,
    'All': 'Total'
}
purpose_vis_dict = {
    'University': 'univ',
    'School': 'sch',
    'Work': 'work',
    'Work sub-tour': 'atwork',
    'Ind-Discretionary': 'idisc',
    'Ind-Maintenance': 'imain',
    'Joint-Maintenance': 'jmain',
    'Joint-Discretionary': 'jdisc',
    'Total': 'Total'
}



tour_mc_vis = tour_mode_choice_calibration_table.copy()
tour_mc_vis.loc[tour_mc_vis['grouped_tour_mode'].isin(['TNC-SINGLE','TNC-SHARED']), 'grouped_tour_mode'] = 'TAXI'
tour_mc_vis = tour_mc_vis[tour_mc_vis['grouped_tour_mode'] != 'All']
tour_mc_vis['id'] = tour_mc_vis['grouped_tour_mode'].apply(lambda x: mode_vis_dict[x])
tour_mc_vis['purpose'] = tour_mc_vis['purpose'].apply(lambda x: purpose_vis_dict[x])
tour_mc_vis = tour_mc_vis.reset_index(drop=True).pivot_table(
    columns='auto_suff', values='tours', index=['id','purpose']).reset_index()
tour_mc_vis = tour_mc_vis.rename(columns={0: 'freq_as0', 1: 'freq_as1', 2: 'freq_as2', 'All': 'freq_all'})
tour_mc_vis.to_csv(os.path.join(settings['visualizer_summaries'],'tmodeProfile_vis_calib.csv'), index=False)



In [79]:


modename_vis_dict = {
    'SOV': 'Auto SOV',
    'HOV2': 'Auto 2 Person',
    'HOV3': 'Auto 3+ Person',
    'WALK': 'Auto SOV',
    'BIKE': 'Bike/Moped',
    'WALK-TRANSIT': 'Walk-Transit',
    'PNR-TRANSIT': 'PNR-Transit',
    'KNR-TRANSIT': 'KNR-Transit',
    'TNC-TRANSIT': 'TNR-Transit',
    'TAXI': 'Taxi',
    'SCHOOLBUS': 'School Bus',
    'All': 'Total'
}
trip_purpose_vis_dict = purpose_vis_dict
trip_purpose_vis_dict['Total'] = 'total'
trip_purpose_vis_dict['School'] = 'schl'
trip_mc_vis = trip_mode_choice_calibration_table.copy()

trip_mc_vis.loc[trip_mc_vis['grouped_linked_trip_mode'].isin(['TNC-SINGLE','TNC-SHARED']), 'grouped_linked_trip_mode'] = 'TAXI'
trip_mc_vis.loc[trip_mc_vis['grouped_tour_mode'].isin(['TNC-SINGLE','TNC-SHARED']), 'grouped_tour_mode'] = 'TAXI'

trip_mc_vis = trip_mc_vis.groupby(['grouped_linked_trip_mode','grouped_tour_mode', 'purpose']).trips.sum().reset_index()


trip_mc_vis = trip_mc_vis[trip_mc_vis['grouped_linked_trip_mode'] != 'All']
trip_mc_vis['tripmode'] = trip_mc_vis['grouped_linked_trip_mode'].apply(lambda x: mode_vis_dict[x])
trip_mc_vis['tourmode'] = trip_mc_vis['grouped_tour_mode'].apply(lambda x: modename_vis_dict[x])

trip_mc_vis['value'] = trip_mc_vis['trips']
trip_mc_vis.loc[trip_mc_vis['tourmode'] == 'All', 'tourmode'] = 'Total'
trip_mc_vis['tourmode_num'] = trip_mc_vis['grouped_tour_mode'].apply(lambda x: mode_vis_dict[x])
trip_mc_vis['purpose'] = trip_mc_vis['purpose'].apply(lambda x: trip_purpose_vis_dict[x])
trip_mc_vis['grp_var'] = trip_mc_vis.apply(lambda row: \
                                           row['purpose'] + 'tourmode' + str(row['tourmode_num'])
                                           if row['tourmode'] != 'Total' \
                                           else  row['purpose'] + str(row['tourmode_num']), axis=1)
trip_mc_vis_cols = ['tripmode', 'tourmode', 'purpose', 'value', 'grp_var']
trip_mc_vis[trip_mc_vis_cols].to_csv(os.path.join(settings['visualizer_summaries'],'tripModeProfile_vis_calib.csv'), index=False)
